Import Packages


In [ ]:

import os
import time
import math
import string
import random

import collections
import numpy as np
import matplotlib.pyplot as plt

import torch
from datasets import load_dataset
from torch import nn
from torch import optim
from torch.functional import F
from torch.utils.data import DataLoader



# Data

In [ ]:
seed = 42
torch.manual_seed(seed)
# Probably, this below must be changed if you work with a M1/M2/M3 Mac
torch.cuda.manual_seed(seed) # for CUDA
torch.backends.cudnn.deterministic = True # for CUDNN
torch.backends.benchmark = False # if True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.

# Q1

In [ ]:
# Question 1
ds = load_dataset("heegyu/news-category-dataset")
print(ds['train'])
print(ds['train'].column_names)


i. dataset is a package providing two main features: the first one is to be a dataloader for many public datasets, and the second one is a way to be efficient in the data pre-processing


ii. The dataset is loaded as an object of the DatasetDict type, provided by Hugging Face Datasets. It is essentially a Python dictionary where: Keys represent dataset splits (e.g., train, test, validation). Values are Dataset objects, which are table-like structures similar to pandas DataFrames or NumPy arrays.


iii. ['link', 'headline', 'category', 'short_description', 'authors', 'date'], in total 6, the first 5 in string and the last one is in s, are fundamental for nlp processing

# Q2

In [ ]:
politics_news = ds['train'].filter(lambda x: x['category'] == 'POLITICS')
print(len(politics_news))


# Q3

In [ ]:
tokent_text = [title.lower() for title in politics_news]